# IMU Feature Extraction Pipeline

In [ ]:
from pathlib import Path
import re

import pandas as pd
import numpy as np

import biopsykit as bp
from biopsykit.utils.time import timedelta_to_time
from biopsykit.signals.imu.feature_extraction.static_sequences import static_sequence_features

import carwatch_analysis.helper_functions as helper

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close('all')
sns.set_theme(style='ticks')

load_raw = True
export_figures = False

In [ ]:
base_path = Path("../../../../Data")
sleep_path = base_path.joinpath("Sleep_Data")

# create new export folders
export_path = Path("../../exports")
feature_export_path = export_path.joinpath("features")
sleep_endpoints_export_path = export_path.joinpath("sleep_endpoints")
plot_export_path = export_path.joinpath("sleep_plots")

bp.utils.file_handling.mkdirs([export_path, plot_export_path, feature_export_path, sleep_endpoints_export_path])

In [ ]:
fs = 102.4
window_size = 5 * fs
overlap = int(0.9 * window_size)
thres = 100

In [ ]:
# get all subject directories
subject_dirs = bp.utils.file_handling.get_subject_dirs(sleep_path, pattern="*")

In [ ]:
feature_results = {}
endpoint_results = {}

for subject_dir in tqdm(subject_dirs):
    subject_id = helper.subject_id_from_path(subject_dir)
    
    df_endpoints_subject, df_features_subject = helper.process_subject(
        subject_dir=subject_dir, 
        load_raw=load_raw, 
        export_figures=export_figures, 
        feature_export_path=feature_export_path, 
        sleep_endpoints_export_path=sleep_endpoints_export_path, 
        plot_export_path=plot_export_path,
        thres=thres, window_size=window_size, overlap=overlap
    )
    
    if df_endpoints is not None:
        feature_results[subject_id] = df_features_subject
        endpoint_results[subject_id] = df_endpoints_subject

In [ ]:
df_features = pd.concat(feature_results, names=['subject'])
df_sleep_endpoints = pd.concat(endpoint_results, names=['subject'])

In [ ]:
df_features.to_csv(export_path.joinpath("imu_features_complete.csv"))
df_sleep_endpoints.to_csv(export_path.joinpath("imu_sleep_endpoints_complete.csv"))

## Tests

In [ ]:
df_feature_plot = df_feature.drop(index=0, level='label').reset_index()
df_feature_plot = df_feature_plot.drop(columns=['subject', 'night', 'sleep_bouts_number'])

In [ ]:
df_feature_plot

In [ ]:
g = sns.pairplot(df_feature_plot, hue='label')

In [ ]:
g.savefig(export_path.joinpath("pairplot.pdf"))